In [1]:
folder = 'mouse-brain-rna-seq-2019/'
filename = folder + 'matrix.csv'
num_rows = 74900 # Five hundred million
batch_size = 5000

In [2]:
#TODO cluster_order codify into 1,2,3,4,5

In [3]:
import numpy as np
import pandas as pd
import csv
import tensorflow as tf

In [4]:
def cutoff_index(out_num):
    #take fraction of cell data with highest variance in trimmed means
    df = pd.read_csv(folder+'trimmed_means.csv')
    #df = pd.read_csv(folder+'medians.csv')
    dg = df.var(axis=0)
    dg_sort = dg.sort_values(ascending=False)
    cutoff = dg_sort[out_num]
    gene_names = dg[dg>cutoff].index.values
    #ngenes = len(gene_names)
    dh=dg.copy(deep=True)
    dh[:]=np.arange(len(dh))
    gene_index = dh[dg>cutoff].values
    return gene_names,gene_index

In [5]:
gene_names,gene_index = cutoff_index(10000)

In [6]:
cell_types=pd.read_csv(folder+'metadata.csv',usecols=['cluster_order'])

In [7]:
len(cell_types['cluster_order'].unique())

367

In [8]:
cell_types[cell_types['cluster_order']>379]

,cluster_order


In [9]:
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(8000, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  #tf.keras.layers.Dense(367)
  tf.keras.layers.Dense(380)
])

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(8000, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(4000, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2000, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  #tf.keras.layers.Dense(367)
  tf.keras.layers.Dense(380)
])

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
# Choose and optimizer and compile a model
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [15]:
datafile = filename
chunksize = batch_size
total_cells=num_rows
trainsize=int(total_cells*0.8)

#models = []
#for chunk in pd.read_csv(datafile, chunksize=chunksize):
for epoch in range(30):
    i=0
    in_chunker = pd.read_csv(datafile, chunksize=chunksize, nrows=trainsize, usecols=gene_names) #skiprows=1
    out_chunker = pd.read_csv(folder+'metadata.csv', chunksize=chunksize, nrows=trainsize,usecols=['cluster_order'])

    test_in_chunker = pd.read_csv(datafile, chunksize=chunksize, skiprows=[trainsize], usecols=gene_names)
    test_out_chunker = pd.read_csv(folder+'metadata.csv', chunksize=chunksize, skiprows=[trainsize],usecols=['cluster_order'])
    
    for in_chunk, out_chunk in zip(in_chunker, out_chunker):
        in_chunk=in_chunk.values.astype(int)
        label_chunk = out_chunk.fillna(0).astype(int)#['cell_type_alias_id']#out_chunk['cell_type_alias_label']
        print(in_chunk.shape,label_chunk.shape)
        model.fit(in_chunk, label_chunk, epochs = 1, batch_size = 100)
        #model.fit(in_chunk, label_chunk, steps_per_epoch=60, epochs=5)#,
                 #steps_per_epoch=trainsize / batch_size, epochs=10, verbose=1)
            #train_on_batch(in_chunk, label_chunk)
        i=i+1
        print(epoch,i,i*chunksize,(i*chunksize)/trainsize)

(5000, 10000) (5000, 1)
50/50 [==============================] - 47s 938ms/step - loss: 1481.4211 - accuracy: 0.0280
0 1 5000 0.08344459279038718
(5000, 10000) (5000, 1)
50/50 [==============================] - 43s 870ms/step - loss: 12.6924 - accuracy: 0.0216
0 2 10000 0.16688918558077437
(5000, 10000) (5000, 1)
50/50 [==============================] - 43s 861ms/step - loss: 8.0018 - accuracy: 0.0290
0 3 15000 0.25033377837116155
(5000, 10000) (5000, 1)
50/50 [==============================] - 44s 881ms/step - loss: 6.7848 - accuracy: 0.0106
0 4 20000 0.33377837116154874
(5000, 10000) (5000, 1)
50/50 [==============================] - 43s 868ms/step - loss: 6.4821 - accuracy: 0.0192
0 5 25000 0.4172229639519359
(5000, 10000) (5000, 1)
50/50 [==============================] - 43s 863ms/step - loss: 5.6665 - accuracy: 0.0468
0 6 30000 0.5006675567423231
(5000, 10000) (5000, 1)
50/50 [==============================] - 44s 872ms/step - loss: 4.8108 - accuracy: 0.0656
0 7 35000 0.584112149

In [16]:
model.save('i10000-8000-4000-2000-o380')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: i10000-8000-4000-2000-o380\assets


In [18]:
test_in_chunker = pd.read_csv(datafile, chunksize=chunksize, skiprows=[trainsize], usecols=gene_names)
test_out_chunker = pd.read_csv(folder+'metadata.csv', chunksize=chunksize, skiprows=[trainsize],usecols=['cluster_order'])

results=[]
i=0
for in_chunk, out_chunk in zip(test_in_chunker, test_out_chunker):
    in_chunk=in_chunk.values.astype(int)
    label_chunk = out_chunk.fillna(0).astype(int)#['cell_type_alias_id']#out_chunk['cell_type_alias_label']
    print(in_chunk.shape,label_chunk.shape)
    r=model.evaluate(in_chunk, label_chunk, batch_size = 100)
    results.append(r)
    print("test loss, test acc:", results[i])
    #model.fit(in_chunk, label_chunk, steps_per_epoch=60, epochs=5)#,
             #steps_per_epoch=trainsize / batch_size, epochs=10, verbose=1)
        #train_on_batch(in_chunk, label_chunk)
    i=i+1
    print(epoch,i,i*chunksize,(i*chunksize)/trainsize)



(5000, 10000) (5000, 1)
50/50 [==============================] - 13s 256ms/step - loss: 5.9526 - accuracy: 0.0000e+00
test loss, test acc: [5.952617168426514, 0.0]
29 1 5000 0.08344459279038718
(5000, 10000) (5000, 1)
50/50 [==============================] - 13s 268ms/step - loss: 6.0037 - accuracy: 0.0000e+00
test loss, test acc: [6.003654479980469, 0.0]
29 2 10000 0.16688918558077437
(5000, 10000) (5000, 1)
50/50 [==============================] - 14s 282ms/step - loss: 4.9118 - accuracy: 0.0658
test loss, test acc: [4.9118499755859375, 0.0658000037074089]
29 3 15000 0.25033377837116155
(5000, 10000) (5000, 1)
50/50 [==============================] - 13s 268ms/step - loss: 5.8555 - accuracy: 2.0000e-04
test loss, test acc: [5.855502128601074, 0.00019999999494757503]
29 4 20000 0.33377837116154874
(5000, 10000) (5000, 1)
50/50 [==============================] - 13s 267ms/step - loss: 5.4872 - accuracy: 0.0442
test loss, test acc: [5.4871954917907715, 0.044199999421834946]
29 5 25000 0

KeyboardInterrupt: 

In [26]:
np.average(results,axis=0)

array([4.80286276, 0.06308333])